In [1]:
#t1
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import wikipedia as wp
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, accuracy_score, precision_score, recall_score
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Wikipedia에서 S&P 500 티커 목록 가져오기
html = wp.page("List of S&P 500 companies").html()
sp500_df = pd.read_html(html)[0]
tickers = sp500_df['Symbol'].tolist()

In [3]:
tickers_to_download = tickers + ['^VIX']

In [4]:
#데이터 다운로드
start_date = '2015-01-01'
end_date = '2020-12-31'
raw_data = yf.download(tickers_to_download, start=start_date, end=end_date, progress=True)

print("데이터 다운로드 완료.")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  504 of 504 completed

10 Failed downloads:
['EXE', 'GEHC', 'SOLV', 'GEV', 'CEG', 'KVUE', 'VLTO', 'COIN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2020-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1609390800")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2020-12-31)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


데이터 다운로드 완료.


In [ ]:
# Open, Close, VIX 데이터만 선택
data_subset = raw_data[['Open', 'Close']]
vix_data = raw_data['Close']['^VIX'].rename('VIX_Close')

# 데이터를 long-format(stacked)으로 변환
stacked_data = data_subset.stack().reset_index()
stacked_data.columns = ['Date', 'Stock', 'Open', 'Close']

# VIX 데이터 병합
stacked_data = pd.merge(stacked_data, vix_data, on='Date', how='left')

# 섹터 및 시가총액 정보 가져오기 (API 호출이 많아 시간이 걸릴 수 있습니다)
metadata = {}
for ticker in tqdm(tickers, desc="Fetching Metadata"):
    try:
        info = yf.Ticker(ticker).info
        metadata[ticker] = {
            'Sector': info.get('sector', 'N/A'),
            'Market_Cap': info.get('marketCap', np.nan)
        }
    except Exception as e:
        print(f"Could not get info for {ticker}: {e}")
        metadata[ticker] = {'Sector': 'N/A', 'Market_Cap': np.nan}

metadata_df = pd.DataFrame.from_dict(metadata, orient='index').reset_index().rename(columns={'index': 'Stock'})

# 주가 데이터에 메타데이터 병합
full_df = pd.merge(stacked_data, metadata_df, on='Stock', how='left')

# 데이터 클리닝
full_df.dropna(subset=['Open', 'Close', 'Sector', 'Market_Cap'], inplace=True)
full_df = full_df[full_df['Sector'] != 'N/A']

print(f"전처리 후 데이터 형태: {full_df.shape}")
print("데이터 샘플:")
print(full_df.head())

Fetching Metadata:   3%|▋                      | 14/503 [00:07<04:18,  1.89it/s]

In [9]:
def feature_engineering(df):
    df.sort_values(by=['Stock', 'Date'], inplace=True)
    df['Stock_Return'] = df.groupby('Stock')['Close'].pct_change().fillna(0)
    
    # 섹터별 평균 수익률 및 비정상 수익률 [cite: 70]
    sector_return = df.groupby(['Date', 'Sector'])['Stock_Return'].transform('mean')
    df['Abnormal_Return'] = df['Stock_Return'] - sector_return
    
    # EMA 피처 [cite: 121]
    # 45개
    print("EMA 피처 생성 중...")
    short_windows = [1, 3, 5, 7, 9, 11, 13, 15, 17]
    long_windows = [3, 5, 7, 9, 11, 13, 15, 17, 19]
    combinations = []
    for s in short_windows:
        for l in long_windows:
        # 롱윈도우가 숏윈도우보다 최소 2일 길어야 한다는 조건
            if l >= s + 2:
                combinations.append((s, l))
    for s, l in tqdm(combinations, desc="Generating 45 EMA Features"):
        col_name = f'EMA_{l}_{s}'
        ema_short = df.groupby('Stock')['Close'].transform(lambda x: ta.ema(x, length=s))
        ema_long = df.groupby('Stock')['Close'].transform(lambda x: ta.ema(x, length=l))
        df[col_name] = (ema_short - ema_long) / ema_long

    # VIX 피처 [cite: 140]
    df['VIX_20'] = ta.ema(df['VIX_Close'], length=20)
    df['VIX_100'] = ta.ema(df['VIX_Close'], length=100)
    df['VIX_Feature'] = df['VIX_20'] - df['VIX_100']

    # 개별 주식 변동성 피처 [cite: 143]
    df['Stock_Vol_5'] = df.groupby('Stock')['Abnormal_Return'].transform(lambda x: x.ewm(span=5).std())
    df['Stock_Vol_20'] = df.groupby('Stock')['Abnormal_Return'].transform(lambda x: x.ewm(span=20).std())
    df['Stock_Unique_Volatility'] = df['Stock_Vol_5'] - df['Stock_Vol_20']

    return df.dropna()

processed_df = feature_engineering(full_df.copy())
print("피처 엔지니어링 완료:")
print(processed_df.head())

EMA 피처 생성 중...



Generating 45 EMA Features:  36%|█████▎         | 16/45 [00:08<00:15,  1.86it/s]Exception ignored in: <function tqdm.__del__ at 0x11c2bf880>
Traceback (most recent call last):
  File "/Library/anaconda3/envs/t1/lib/python3.13/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/Library/anaconda3/envs/t1/lib/python3.13/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'

Generating 45 EMA Features: 100%|███████████████| 45/45 [00:25<00:00,  1.75it/s]


피처 엔지니어링 완료:
            Date Stock       Open      Close  VIX_Close      Sector  \
46173 2015-05-27     A  38.859116  39.171680      13.27  Healthcare   
46640 2015-05-28     A  39.024577  38.381062      13.31  Healthcare   
47107 2015-05-29     A  38.417859  37.866272      13.84  Healthcare   
47574 2015-06-01     A  38.086902  37.618053      13.97  Healthcare   
48041 2015-06-02     A  37.636437  37.792721      14.24  Healthcare   

         Market_Cap  Stock_Return  Abnormal_Return   EMA_3_1  ...  EMA_19_13  \
46173  3.518802e+10      0.013076         0.001648  0.004184  ...   0.000781   
46640  3.518802e+10     -0.020183        -0.019030 -0.008107  ...   0.000141   
47107  3.518802e+10     -0.013413        -0.016788 -0.010821  ...  -0.000926   
47574  3.518802e+10     -0.006555        -0.011470 -0.008728  ...  -0.002017   
48041  3.518802e+10      0.004643         0.010132 -0.002067  ...  -0.002639   

       EMA_17_15  EMA_19_15  EMA_19_17     VIX_20    VIX_100  VIX_Feature  \
46

In [10]:
# ===== [수정된 셀 7] =====

def quantile_binning(df):
    # 'EMA_'가 포함된 모든 컬럼 이름을 가져옵니다.
    ema_cols = [col for col in df.columns if 'EMA_' in col]
    # Market_Cap과 Stock_Unique_Volatility는 VIX와 함께 별도 처리하거나 논문 방식에 따라 처리합니다.
    # 여기서는 논문의 핵심인 EMA 피처 이산화에 집중합니다.
    features_to_bin = ema_cols # + ['Stock_Unique_Volatility', 'Market_Cap']
    
    print("Quantile Binning 중 (EMA Features)...")
    for col in tqdm(features_to_bin, desc="Binning EMA features"):
        # <<수정 포인트 1>>: 그룹화 기준에 'Sector'를 추가하여 섹터 내 분위수를 계산합니다.
        df[col] = df.groupby(['Date', 'Sector'])[col].transform(
            lambda x: pd.qcut(x, 5, labels=False, duplicates='drop')
        )
    return df

# VIX 피처는 아래 다음 셀에서 별도로 처리하므로, 여기서는 EMA만 처리합니다.
binned_df = quantile_binning(processed_df.copy())

# 타겟 변수 생성
binned_df['Target'] = (binned_df['Abnormal_Return'] > 0).astype(int)

# 데이터 분할
train_df = binned_df[pd.to_datetime(binned_df['Date']) < '2020-01-01']
test_df = binned_df[pd.to_datetime(binned_df['Date']) >= '2020-01-01']

print(f"학습 데이터: {train_df.shape}, 테스트 데이터: {test_df.shape}")

Quantile Binning 중...


Binning features: 100%|█████████████████████████| 48/48 [01:43<00:00,  2.17s/it]


학습 데이터: (590344, 61), 테스트 데이터: (123384, 61)


In [ ]:
# ===== [새로 추가할 셀] =====

print("VIX 피처 롤링 윈도우 이산화 중...")

# 1. 날짜별 고유 VIX 값을 추출하여 시계열 데이터 생성
vix_daily = binned_df[['Date', 'VIX_Close']].drop_duplicates().set_index('Date').sort_index()

# 2. 30일 롤링 윈도우를 적용하여 VIX 값을 5분위수로 변환하는 함수
def assign_rolling_quintile(series):
    current_value = series.iloc[-1]
    try:
        bins = pd.qcut(series, 5, retbins=True, duplicates='drop')[1]
        return pd.cut([current_value], bins=bins, labels=False, include_lowest=True)[0]
    except (ValueError, IndexError):
        return np.nan

# 3. 함수 적용
vix_daily['VIX_quintile'] = vix_daily['VIX_Close'].rolling(window=30, min_periods=5).apply(assign_rolling_quintile, raw=False)

# 4. 계산된 VIX 분위수 값을 원래 데이터프레임에 병합
# 기존의 VIX_Feature 대신 새로운 VIX_quintile을 사용합니다.
train_df = pd.merge(train_df, vix_daily[['VIX_quintile']], on='Date', how='left')
test_df = pd.merge(test_df, vix_daily[['VIX_quintile']], on='Date', how='left')

# 병합 후 생길 수 있는 결측치 처리
train_df.dropna(subset=['VIX_quintile'], inplace=True)
test_df.dropna(subset=['VIX_quintile'], inplace=True)
train_df['VIX_quintile'] = train_df['VIX_quintile'].astype(int)
test_df['VIX_quintile'] = test_df['VIX_quintile'].astype(int)


print("VIX 피처 이산화 및 병합 완료.")
print("VIX_quintile 컬럼이 추가되었습니다.")
print(train_df[['Date', 'VIX_Close', 'VIX_quintile']].head())

In [11]:
def ssfi_feature_selection(df, features):
    sample_weights = np.abs(df['Abnormal_Return'])
    y = df['Target']
    cv = KFold(n_splits=10, shuffle=False)
    
    feature_scores = {}
    for feature in tqdm(features, desc="SSFI Feature Selection"):
        X = df[[feature]]
        scores = []
        
        # CV를 통한 성능 측정
        for train_idx, val_idx in cv.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            sw_train, sw_val = sample_weights.iloc[train_idx], sample_weights.iloc[val_idx]
            
            model = BaggingClassifier(
                DecisionTreeClassifier(), 
                n_estimators=100, # 계산 시간을 위해 estimators 수 조정
                random_state=42
            )
            model.fit(X_train, y_train, sample_weight=sw_train)
            preds = model.predict(X_val)
            scores.append(matthews_corrcoef(y_val, preds, sample_weight=sw_val))
        
        feature_scores[feature] = np.mean(scores)

    return sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)[:10]

ema_features = [col for col in binned_df.columns if 'EMA_' in col]
top_10_features = ssfi_feature_selection(train_df, ema_features)
selected_features = [f[0] for f in top_10_features]

print("\n상위 10개 피처:")
print(selected_features)

SSFI Feature Selection:  44%|████████▍          | 20/45 [22:53<28:37, 68.69s/it]


KeyboardInterrupt: 

In [ ]:
# ===== [수정된 셀 9] =====

# <<수정 포인트 2>>: Sector를 정수 레이블로 인코딩합니다.
train_df['Sector_encoded'] = pd.factorize(train_df['Sector'])[0]
test_df['Sector_encoded'] = pd.factorize(test_df['Sector'])[0]


# additional_features에 새로 만든 VIX_quintile과 Sector_encoded를 사용합니다.
# 기존 VIX_Feature와 Market_Cap은 논문 핵심 피처가 아니므로 일단 제외하거나 필요시 추가합니다.
additional_features = ['Sector_encoded', 'VIX_quintile', 'Stock_Unique_Volatility']

# 피처셋 정의
train_features = train_df[selected_features + additional_features]
test_features = test_df[selected_features + additional_features]

# 학습/테스트 데이터의 컬럼을 동일하게 맞춤
train_labels, test_labels = train_features.align(test_features, join='inner', axis=1, fill_value=0)

X_train_primary = train_labels[selected_features]
X_test_primary = test_labels[selected_features]

X_train_add = train_labels[[col for col in additional_features if col in train_labels.columns]]
X_test_add = test_labels[[col for col in additional_features if col in test_labels.columns]]

y_train = train_df.loc[train_labels.index, 'Target']
y_test = test_df.loc[test_labels.index, 'Target']
sw_train = np.abs(train_df.loc[train_labels.index, 'Abnormal_Return'])
sw_test = np.abs(test_df.loc[test_labels.index, 'Abnormal_Return'])


# <<수정 포인트 3>>: 모델 파라미터를 min_weight_fraction_leaf로 변경합니다.
rf_params = {
    'n_estimators': 200, 
    'max_features': 0.5, 
    'min_weight_fraction_leaf': 0.001,  # 샘플 개수가 아닌 가중치 합의 비율을 기준으로 리프 노드 결정
    'random_state': 42,
    'n_jobs': -1
}

# 1. Primary Model 학습
print("Primary Model 학습 중...")
primary_model = RandomForestClassifier(**rf_params)
primary_model.fit(X_train_primary, y_train, sample_weight=sw_train)
primary_preds_train = primary_model.predict(X_train_primary)

# Meta Target 생성
meta_target_train = (primary_preds_train == y_train).astype(int)

# 2. Meta Model 1 (Regimes Only) 학습
print("Meta Model 1 학습 중...")
meta_model_1 = RandomForestClassifier(**rf_params)
meta_model_1.fit(X_train_add, meta_target_train, sample_weight=sw_train)

# 3. Meta Model 2 (Regimes + X) 학습
print("Meta Model 2 학습 중...")
meta_model_2 = RandomForestClassifier(**rf_params)
meta_model_2.fit(train_labels, meta_target_train, sample_weight=sw_train)

# 4. Non-Meta Model 학습
print("Non-Meta Model 학습 중...")
non_meta_model = RandomForestClassifier(**rf_params)
non_meta_model.fit(train_labels, y_train, sample_weight=sw_train)

print("\n모든 모델 학습 완료.")

In [ ]:
def evaluate_model(y_true, y_pred, sample_weight):
    mcc = matthews_corrcoef(y_true, y_pred, sample_weight=sample_weight)
    accuracy = accuracy_score(y_true, y_pred, sample_weight=sample_weight)
    precision_1 = precision_score(y_true, y_pred, sample_weight=sample_weight, pos_label=1)
    precision_0 = precision_score(y_true, y_pred, sample_weight=sample_weight, pos_label=0)
    return {
        'Matthews': mcc, 
        'Accuracy': accuracy, 
        'Precision 1': precision_1, 
        'Precision 0': precision_0
    }

print("--- OOS 평가 결과 ---")

# Primary Model 평가
primary_preds_test = primary_model.predict(X_test_primary)
primary_results = evaluate_model(y_test, primary_preds_test, sw_test)
print("\nPrimary Model 결과:\n", pd.Series(primary_results))

# Non-Meta Model 평가
non_meta_preds_test = non_meta_model.predict(test_labels)
non_meta_results = evaluate_model(y_test, non_meta_preds_test, sw_test)
print("\nNon-Meta Model 결과:\n", pd.Series(non_meta_results))

# Meta-labeling 적용 후 평가
meta_preds_1 = meta_model_1.predict(X_test_add)
meta_preds_2 = meta_model_2.predict(test_labels)

# Meta Model 1
y_test_meta1 = y_test[meta_preds_1 == 1]
primary_preds_meta1 = pd.Series(primary_preds_test, index=y_test.index)[meta_preds_1 == 1]
sw_test_meta1 = sw_test[meta_preds_1 == 1]
if not y_test_meta1.empty:
    meta1_results = evaluate_model(y_test_meta1, primary_preds_meta1, sw_test_meta1)
    print("\nMeta Model 1 ( 필터링 후) 결과:\n", pd.Series(meta1_results))

# Meta Model 2
y_test_meta2 = y_test[meta_preds_2 == 1]
primary_preds_meta2 = pd.Series(primary_preds_test, index=y_test.index)[meta_preds_2 == 1]
sw_test_meta2 = sw_test[meta_preds_2 == 1]
if not y_test_meta2.empty:
    meta2_results = evaluate_model(y_test_meta2, primary_preds_meta2, sw_test_meta2)
    print("\nMeta Model 2 ( 필터링 후) 결과:\n", pd.Series(meta2_results))